<a href="https://www.kaggle.com/code/estopadilla/market-purchase-analyst-enterprise-ai-agent?scriptVersionId=283418629" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🚨 Problem to solve

There is a widespread problem within companies dedicated to trade with any kind of products, companies that typically act as intermediaries between suppliers and the end customer. The core of the business is focused on profiting from these transactions, constantly searching through different suppliers or competitors for different offers, buying when they believe they can sell at a price high enough to make a profit.

The problem arises from the **complexity of analyzing** each and every supplier or store that offers each product, making it tedious (and almost impossible) to cover all possible purchasing options for a given product. This AI-based solution addresses this issue. In a simple, effective, and natural way, this tool allows users to search for information on any product online, simplifying this task to an extreme degree.

- Each query automatically returns the best offer for each product, as well as relevant information such as delivery time, supplier, and product status. It also allows filtering by product status.

- It will also provide the user with information on the current price, indicating the price trend, its position relative to historical data, and even the highest and lowest prices reached by that supplier for that product.

- After obtaining this information, the user will also be given the option to quickly purchase the product, specifying the desired condition and quantity. By doing this, the user will be obtain all necessary information to make a purchase.

With this workflow, **a complete comparison of suppliers, prices, and market flows can be performed for a single product, and a purchase order can be placed with the desired conditions, all in a matter of seconds.**

It is worth noting that everything will be simulated using local databases and methods, with the ability to connect to different agents or APIs to replicate this scenario in real-world environments.

# ⚙️ Scenario setup

## 👉 All imports and key addition

First of all, let's importing all necessary libraries for the project, also adding the GOOGLE_API_KEY in order to use google provided tools.

In [1]:
# Imports for google api key instantiation
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Setup and authentication complete.


In [2]:
# Imports for mock database creation
import sqlite3
import json
import random

# Import for agents creation and management
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool#, FunctionTool
from google.adk.tools.function_tool import FunctionTool
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.tool_context import ToolContext
import uuid

# Avoid additional warnings
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

# Ensure that project root is the correct one
import sys
if "/kaggle/working" not in sys.path:
    sys.path.insert(0, "/kaggle/working")
    print("🔶 Project root /kaggle/working needed and added")

# Initial vars
DB_NAME = "mock_products.db"
APP_NAME = "Marker_Purchase_Manager"
USER_ID = "enterprise_user"

print("✅ Initial setup complete.")

✅ Initial setup complete.


## 👉 Synthetic data and price fluctuation

First of all, let's create a database with a synthetic information, in order to simulate that this information is over internet or behind an API. This database will not be necessary when using a real API.

In [3]:
# Database creation and definition
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_name TEXT,
            product_vendor TEXT,
            product_specs TEXT,
            last_checked_prices TEXT,
            delivery_available INTEGER,
            delivery_days INTEGER,
            delivery_fee REAL,
            status TEXT
        );
    """)

    conn.commit()
    conn.close()

print("✅ Function to create the database created.")

✅ Function to create the database created.


In [4]:
# Function to insert a single product into database
def insert_product(entry: str):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO products 
        (product_name, product_vendor, product_specs, last_checked_prices,
         delivery_available, delivery_days, delivery_fee, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        entry["product_name"],
        entry["product_vendor"],
        entry["product_specs"],
        json.dumps(entry["last_checked_prices"]),
        1 if entry["delivery_available"] else 0,
        entry["delivery_days"],
        entry["delivery_fee"],
        entry["status"]
    ))

    conn.commit()
    conn.close()

print("✅ Function to insert each element into database created.")

✅ Function to insert each element into database created.


In [5]:
# Function to fill the database with invented data
def seed_database():
    devices = [
        ("iPhone 13", "128GB, OLED display, A15 Bionic"),
        ("iPhone 14 Pro", "256GB, ProMotion 120Hz"),
        ("Samsung Galaxy S22", "128GB, Snapdragon 8 Gen 1"),
        ("Samsung Galaxy S23 Ultra", "512GB, S-Pen"),
        ("Google Pixel 7", "128GB, Tensor G2"),
        ("Google Pixel 8 Pro", "256GB, Tensor G3"),
        ("MacBook Air M1", "8GB RAM, 256GB SSD"),
        ("MacBook Air M2", "16GB RAM, 512GB SSD"),
        ("MacBook Pro M3", "18GB RAM, 1TB SSD"),
        ("iPad Pro M2", "11-inch, 128GB"),
        ("Dell XPS 13", "16GB RAM, 512GB SSD"),
        ("Lenovo ThinkPad X1 Carbon", "16GB RAM, 1TB SSD"),
        ("Asus ROG Zephyrus G14", "RTX 4060, 16GB RAM"),
        ("Sony WH-1000XM4", "ANC headphones"),
        ("Sony WH-1000XM5", "ANC headphones"),
        ("Bose QC45", "Noise cancelling headphones"),
        ("Logitech MX Master 3", "Ergonomic mouse, USB-C"),
        ("Logitech G Pro X", "Gaming headset"),
        ("Razer Viper Ultimate", "Wireless gaming mouse"),
        ("SteelSeries Arctis 7", "Wireless gaming headset"),
        ("Samsung 970 EVO Plus 1TB", "NVMe SSD"),
        ("Samsung 990 Pro 2TB", "NVMe SSD"),
        ("Crucial MX500 1TB", "SATA SSD"),
        ("WD Black SN850X", "NVMe SSD"),
        ("NVIDIA RTX 3060", "12GB GDDR6"),
        ("NVIDIA RTX 4070", "12GB GDDR6X"),
        ("NVIDIA RTX 4090", "24GB GDDR6X"),
        ("AMD Radeon 6800 XT", "16GB GDDR6"),
        ("AMD Ryzen 7 5800X3D", "8-core CPU"),
        ("Intel Core i9 13900K", "24-core CPU"),
        ("Oculus Quest 2", "VR headset"),
        ("Meta Quest 3", "Next-gen VR headset"),
        ("Amazon Kindle Paperwhite", "E-ink display"),
        ("Kindle Oasis", "Premium e-reader"),
        ("LG C2 OLED 55", "4K OLED TV"),
        ("Samsung Odyssey G7", "27'' QHD 240Hz monitor"),
        ("LG 27GP850", "27'' 165Hz gaming monitor"),
        ("Dell UltraSharp U2723QE", "27'' 4K IPS display"),
    ]

    statuses = {
        "new": (200, 2200),
        "good_state": (120, 1500),
        "used": (80, 1000),
        "bad_state": (20, 400),
        "for_pieces": (5, 100),
    }

    vendors = [
        "TechStore", "GigaBuy", "SecondLifeElectronics", "SoundHub", "AudioResale",
        "ScrapTech", "ReTech", "Parts&Co", "GPUPlanet", "AppleWorld",
        "MobileHub", "MegaHardware", "CompuTrade", "ElectroZone", "HyperTech",
        "SmartBuy", "DealHub", "DiscountTech", "FutureElectronics", "NeoTech",
        "TechMart", "ScreenShop", "VRPlanet", "DigitalStation", "ElectronixPro",
        "ComputerPlaza", "FireTech", "SmartGizmos", "DeviceLab", "TechTrade",
        "EuTech", "DigitalDen", "SiliconWorld", "GizmoDepot", "TechieStore",
        "GadgetHouse", "ElectronHub", "ProTechDeals", "ChipMasters"
    ]

    all_entries = []

    for name, specs in devices:
        
        name = name.lower()
        
        for status, price_range in statuses.items():
            for _ in range(3):  
                base = random.uniform(*price_range)

                entry = {
                    "product_name": name,
                    "product_vendor": random.choice(vendors),
                    "product_specs": specs,
                    "last_checked_prices": [
                        round(base + random.uniform(-15, 15), 2),
                        round(base + random.uniform(-15, 15), 2)
                    ],
                    "delivery_available": random.choice([True, False]),
                }

                if entry["delivery_available"]:
                    entry["delivery_days"] = random.randint(1, 14)
                    entry["delivery_fee"] = round(random.uniform(0, 40), 2)
                else:
                    entry["delivery_days"] = -1
                    entry["delivery_fee"] = -1

                entry["status"] = status
                all_entries.append(entry)

    for e in all_entries:
        insert_product(e)

    print(f"Inserted {len(all_entries)} entries into database.")

print("✅ Function to feed the database created.")

✅ Function to feed the database created.


To make the data more realistic, I'll create a function that simulates (randomly generated) new prices for each product in any state (all entries in the database). This way, the scenario will change, and different suppliers will be able to offer a better price at any time.

In [6]:
# Function to simulate price fluctuation over time
def apply_price_fluctuation():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("SELECT id, last_checked_prices FROM products")
    rows = cursor.fetchall()

    for product_id, prices_json in rows:
        prices = json.loads(prices_json)

        last_price = prices[-1]
        fluctuated_price = round(last_price + random.uniform(-20, 20), 2)
        if fluctuated_price < 0.01:
            fluctuated_price = 0.01
        prices.append(fluctuated_price)

        cursor.execute("""
            UPDATE products SET last_checked_prices = ? WHERE id = ?
        """, (json.dumps(prices), product_id))

    conn.commit()
    conn.close()
    #print("Market fluctuation applied.")


print("✅ Function to simulate price fluctuation over time created (give more realistic scenario)")

✅ Function to simulate price fluctuation over time created (give more realistic scenario)


In [7]:
create_database()
seed_database()
apply_price_fluctuation()

print("✅ Database created, feeded and first price fluctuation applied.")

Inserted 570 entries into database.
✅ Database created, feeded and first price fluctuation applied.


# 🤖 Market analyst agent creation

Once the database is created, it's time to create the first agent.

This agent searches all available options for a product (suppliers, other sellers, etc.) in the desired state or in all available states. Once found, it will also monitor the price fluctuations of that product over time, briefly returning the product's state and how good the current price is.

For that, I'll create two functions that will work as tools for the agent. One for a product in a given state and another that retrieves information about all statuses of the given product. For both tools, they will be focused on the cheaper option (the final purpose of this agent is to know the lower price in order to sell the product if proceed).

## 🛠 Tools creation

The initial tools created will be:

- get_product_summary_all_statuses(product_name: str) -> It simulates obtain the best offer of a product looking over internet, returning detailed information about price and its fluctuation, apart from the vendor and specifications of the product. This information is also detailed for different product statuses, like 'new' or 'used' among other.
  
- get_product_summary(product_name: str, status: str) -> It works like the previous one, but returning these information only for a given status. Clearly, it is a specific case of the previous function and can be integrated by defining the functionality when the state is not defined or is "all" for example, but it will be kept separate to better understand the interaction between tools (learning purpose).

In [8]:
# Method to return all available information of the product with a given status 
def get_product_summary(product_name: str, status: str):
    """Obtain the current features of a product and status given.

    This tool simulates looking up over internet and find the best option of a product in a certain status and returns the most relevant information about it.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".
        status: The status of the product to find information about it.

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "product": Samsung Galaxy S22,
            "vendor": TechMart,
            "actual_price": 158.25,
            "price_trend_vs_previous": cheaper,
            "is_lowest_price_ever": False,
            "is_highest_price_ever": True,
            "highest_price_ever": 199.99,
            "lowest_price_ever": 155.58
        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """    

    product_name = product_name.lower()
    status = status.lower()
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT product_name, product_vendor, last_checked_prices, status
        FROM products
        WHERE product_name = ? AND status = ?
    """, (product_name, status))

    rows = cursor.fetchall()
    conn.close()

    apply_price_fluctuation() # To simulate market drift each time this function/tool is invoked

    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found or incorrect status, try another product (for example, Samsung Galaxy S22) or status [new, good_state, used, bad_state, for_pieces]"
        }

    # Convert rows to objects
    entries = []
    for name, vendor, prices_json, st in rows:
        prices = json.loads(prices_json)
        entries.append({
            "product_name": name,
            "product_vendor": vendor,
            "prices": prices,
            "actual_price": prices[-1]
        })

    # Get the cheapest current entry
    cheapest_entry = min(entries, key=lambda x: x["actual_price"])
    prices = cheapest_entry["prices"]

    actual_price = prices[-1]
    previous_price = prices[-2] if len(prices) > 1 else actual_price

    trend = (
        "cheaper" if actual_price < previous_price else
        "expensive" if actual_price > previous_price else
        "same"
    )

    min_price = min(prices)
    max_price = max(prices)

    summary = {
        "status": "success",
        "product": cheapest_entry["product_name"],
        "vendor": cheapest_entry["product_vendor"],
        "actual_price": actual_price,
        "price_trend_vs_previous": trend,
        "is_lowest_price_ever": actual_price == min_price,
        "is_highest_price_ever": actual_price == max_price,
        "highest_price_ever": max_price,
        "lowest_price_ever": min_price
    }

    return summary

print("✅ Tool function to handle product requests based on product status.")

✅ Tool function to handle product requests based on product status.


In [9]:
# Method to return all available information of the product in all available statuses
def get_product_summary_all_statuses(product_name: str):
    """Obtain the current features of a product and status given.

    This tool simulates looking up over internet and find the best option of a product in all status options and returns the most relevant information about it.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "new":{
                "product": Samsung Galaxy S22,
                "vendor": TechMart,
                "actual_price": 158.25,
                "price_trend_vs_previous": cheaper,
                "is_lowest_price_ever": False,
                "is_highest_price_ever": True,
                "highest_price_ever": 199.99,
                "lowest_price_ever": 155.58
            }
            "used": {
            ...
            }

        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """    

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    product_name = product_name.lower()
    
    cursor.execute("""
        SELECT product_name, product_vendor, last_checked_prices, status
        FROM products
        WHERE product_name = ?
    """, (product_name,))

    rows = cursor.fetchall()
    conn.close()
    
    apply_price_fluctuation() # To simulate market drift each time this function/tool is invoked
    
    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found, try another one (for example, Samsung Galaxy S22)"
        }

    # Organize rows by status
    status_groups = {}
    for name, vendor, prices_json, status in rows:
        prices = json.loads(prices_json)

        entry = {
            "product_name": name,
            "product_vendor": vendor,
            "prices": prices,
            "actual_price": prices[-1]
        }

        status_groups.setdefault(status, []).append(entry)

    summaries = {"status": "success"}

    # Build summary per status
    for status, entries in status_groups.items():
        # pick entry with cheapest actual price
        cheapest_entry = min(entries, key=lambda x: x["actual_price"])
        prices = cheapest_entry["prices"]

        actual_price = prices[-1]
        previous_price = prices[-2] if len(prices) > 1 else actual_price

        trend = (
            "cheaper" if actual_price < previous_price else
            "expensive" if actual_price > previous_price else
            "same"
        )

        min_price = min(prices)
        max_price = max(prices)

        summaries[status] = {
            "product": cheapest_entry["product_name"],
            "vendor": cheapest_entry["product_vendor"],
            "actual_price": actual_price,
            "price_trend_vs_previous": trend,
            "is_lowest_price_ever": actual_price == min_price,
            "is_highest_price_ever": actual_price == max_price,
            "lowest_price_ever": min_price,
            "highest_price_ever": max_price
        }

    return summaries

print("✅ Tool function to handle product requests in all status options.")

✅ Tool function to handle product requests in all status options.


In [10]:
# Quick tests for both functions / tools

print("🔶 Quick test (existing product) --> Summary for: NVIDIA RTX 3060 in used state")
print(get_product_summary("NVIDIA RTX 3060", "used"))
print(get_product_summary_all_statuses("NVIDIA RTX 3060"))

print("🔶 Quick test (non-existing product)--> Summary for: Nintendo Switch 3 in new state")
print(get_product_summary("Nintendo Switch 3", "new"))
print(get_product_summary_all_statuses("Nintendo Switch 3"))

🔶 Quick test (existing product) --> Summary for: NVIDIA RTX 3060 in used state
{'status': 'success', 'product': 'nvidia rtx 3060', 'vendor': 'Parts&Co', 'actual_price': 298.76, 'price_trend_vs_previous': 'expensive', 'is_lowest_price_ever': False, 'is_highest_price_ever': True, 'highest_price_ever': 298.76, 'lowest_price_ever': 261.46}
{'status': 'success', 'new': {'product': 'nvidia rtx 3060', 'vendor': 'GadgetHouse', 'actual_price': 451.48, 'price_trend_vs_previous': 'expensive', 'is_lowest_price_ever': False, 'is_highest_price_ever': False, 'lowest_price_ever': 444.78, 'highest_price_ever': 453.29}, 'good_state': {'product': 'nvidia rtx 3060', 'vendor': 'VRPlanet', 'actual_price': 231.0, 'price_trend_vs_previous': 'cheaper', 'is_lowest_price_ever': True, 'is_highest_price_ever': False, 'lowest_price_ever': 231.0, 'highest_price_ever': 261.12}, 'used': {'product': 'nvidia rtx 3060', 'vendor': 'Parts&Co', 'actual_price': 293.84, 'price_trend_vs_previous': 'cheaper', 'is_lowest_price_e

## 🧠 Agent configuration and definition

Having both tools created, is time to create the agent itself with the tools.

In [11]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)
print("✅ Retry configuration for the model created.")

✅ Retry configuration for the model created.


In [12]:
# Market analyst agent with custom function tools
market_analyst_agent = LlmAgent(
    name="market_analyst_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart product consulter.

    For product status requests:
    1. If user provide product status, use "get_product_summary" tool to get the information related to a product in certain state 
    2. If user don't provide product status or don't mind about it, use "get_product_summary_all_statuses" tool to get infromation related to a product in each state
    3. Check the "status" field in the response of the tool for errors
    4. Return the detailed information obtained explained clearly for the final user.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_product_summary, get_product_summary_all_statuses],
)

print("✅ Market analyst agent created")
print("🔧 Available tools:")
print("  • get_product_summary - Looks up for a product and given status, retrieving relevant information")
print("  • get_product_summary_all_statuses - Looks up for a product in all statuses, retrieving relevant information")

✅ Market analyst agent created
🔧 Available tools:
  • get_product_summary - Looks up for a product and given status, retrieving relevant information
  • get_product_summary_all_statuses - Looks up for a product in all statuses, retrieving relevant information


In [13]:
# Create the runner in order to run the agent
market_analyst_agent_runner = InMemoryRunner(agent=market_analyst_agent)
print("✅ Runner for the market analyst agent created")

✅ Runner for the market analyst agent created


## 👁 Quick model testing

In [15]:
print("🔵 Testing with an existing product and state given")
_ = await market_analyst_agent_runner.run_debug(
    "Can I have some information about the iPhone 14 Pro product in new state?"
)

🔵 Testing with an existing product and state given

 ### Continue session: debug_session_id

User > Can I have some information about the iPhone 14 Pro product in new state?
market_analyst_agent > The iPhone 14 Pro is currently available in "new" condition from the vendor "DealHub". The actual price is $375.78. This is the highest price it has ever been, and it is more expensive compared to previous prices. The lowest price it has ever been is $342.14.


In [17]:
print("🔵 Testing with an existing product without giving state")
_ = await market_analyst_agent_runner.run_debug(
    "Can I have some information about the Google Pixel 8 Pro at any state?"
)

🔵 Testing with an existing product without giving state

 ### Continue session: debug_session_id

User > Can I have some information about the Google Pixel 8 Pro at any state?
market_analyst_agent > The Google Pixel 8 Pro is available in several states:

*   **New:** From "SmartBuy" at $821.10. This is more expensive than previous prices, and it's not the lowest it's ever been ($804.27), but it's also not the highest it's ever been ($841.46).
*   **Used:** From "AppleWorld" at $180.21. This is cheaper than previous prices, and it is currently the lowest price ever for this item.
*   **Good State:** From "TechTrade" at $684.42. This is cheaper than previous prices, but not the lowest or highest it's ever been (lowest: $663.18, highest: $691.96).
*   **Bad State:** From "TechStore" at $65.80. This is the highest price it has ever been.
*   **For Pieces:** From "ChipMasters" at $15.41. This is more expensive than previous prices, and not the lowest or highest it's ever been (lowest: $0.01

In [19]:
print("🔵 Testing with a product that does not exist and state given")
_ = await market_analyst_agent_runner.run_debug(
    "Can I have some information about the Nintendo Switch 3 product in used state?"
)

🔵 Testing with a product that does not exist and state given

 ### Continue session: debug_session_id

User > Can I have some information about the Nintendo Switch 3 product in used state?
market_analyst_agent > I am sorry, but I could not find information about the Nintendo Switch 3 in the 'used' state. Please check the product name and the status, and try again. You can select one of the following statuses: 'new', 'good_state', 'used', 'bad_state', or 'for_pieces'.


In [21]:
print("🔵 Testing with a product that does not exist without giving state")
_ = await market_analyst_agent_runner.run_debug(
    "Can I have some information about the Nintendo Switch 3 product?"
)

🔵 Testing with a product that does not exist without giving state

 ### Continue session: debug_session_id

User > Can I have some information about the Nintendo Switch 3 product?
market_analyst_agent > I am sorry, but I could not find any information about the Nintendo Switch 3. Please check the product name and try again.


# 🤖 Improve agent: Add purchase management

At this point, we already have an operational agent with the necessary tools to perform the function of market analyst. 

In order to achieve the final goal, it is necessary to add one more tool that handles the purchase functionality. Using the current agent as a start point, once it returns the product's features, it will be able to handle all product purchase process.

For that, I'll create a new function that will work as another tool for the agent. Based on a product in a given state (assumes new if not specified), it obtains information related delivery time or fee expected among other features, giving details of a possible purchase.

## 🛠 Tool creation 

The new tool created:

- get_shopping_order_information(product_name: str, status: str, quantity: int) -> It receives the name of the product, the status and the quantity of products to buy. The function simulates accessing to the API and returns all information related to a purchase operation, like delivery time expected or total amout taking into account possible delivery fee. 

In [22]:
# Method to return all available information of purchase a product (simulates placing a purchase order)
def get_shopping_order_information(product_name: str, status: str, quantity: int):

    """Place a shopping order of a product and swith a number of items.

    This tool simulates a product order, calculating the price, the delivery fee and the delivery expected time.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".
        status: The status of the product to place the order.
        quantity: The quantity of items to purchase.

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "item_status": new
            "product": Samsung Galaxy S22,
            "vendor": TechMart,
            "unit_price": 158.25,
            "quantity": quantity,
            "total_price": total_price,
            "delivery_available": "True",
            "delivery_fee": 0,
            "expected_delivery_days": 3
        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """  

    product_name = product_name.lower()
    status = status.lower()        

    if not isinstance(quantity, int) or quantity < 1:
        return {
            "status": "error", 
            "error_message": "Quantity needs to be a valid number and at greather than 0"
        }
        

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT last_checked_prices, delivery_available, delivery_days,
               delivery_fee, product_vendor
        FROM products
        WHERE product_name = ? AND status = ?
    """, (product_name, status))

    rows = cursor.fetchall()
    conn.close()

    apply_price_fluctuation() # To simulate market drift each time this function/tool is invoked

    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found or incorrect status, try another product (for example, Samsung Galaxy S22) or status [new, good_state, used, bad_state, for_pieces]"
        }

    # Convert DB rows into structured objects
    entries = []
    for prices_json, delivery_av, delivery_days, delivery_fee, vendor in rows:
        prices = json.loads(prices_json)

        entries.append({
            "actual_price": prices[-1],
            "delivery_available": delivery_av == 1,
            "delivery_days": delivery_days,
            "delivery_fee": delivery_fee,
            "vendor": vendor,
        })

    # Select the cheapest entry by current price
    best = min(entries, key=lambda x: x["actual_price"])

    # Compute totals
    unit_price = best["actual_price"]
    total_price = round(unit_price * quantity, 2)

    return {
        "status": "success",
        "product": product_name,
        "product_status": status,
        "vendor": best["vendor"],
        "unit_price": unit_price,
        "quantity": quantity,
        "total_price": total_price,
        "delivery_available": best["delivery_available"],
        "delivery_fee": best["delivery_fee"] if best["delivery_available"] else 0,
        "expected_delivery_days": best["delivery_days"] if best["delivery_available"] else None
    }

print("✅ Tool function to place purchase orders created.")

✅ Tool function to place purchase orders created.


In [23]:
# Quick tests for the new function / tool

print("🔶 Quick test (existing product) --> Summary for: NVIDIA RTX 3060 in used state")
print(get_shopping_order_information("NVIDIA RTX 3060", "used", 8))

print("🔶 Quick test (non-existing product)--> Summary for: Nintendo Switch 3 in new state")
print(get_shopping_order_information("Nintendo Switch 3", "new", 3))

print("🔶 Quick test (existing product) but with a bad state --> Summary for: NVIDIA RTX 3060 in broken state")
print(get_shopping_order_information("NVIDIA RTX 3060", "used", 8))

print("🔶 Quick test (existing product) but with an invalid quantity --> Summary for: NVIDIA RTX 3060 in broken state")
print(get_shopping_order_information("NVIDIA RTX 3060", "used", 0))

🔶 Quick test (existing product) --> Summary for: NVIDIA RTX 3060 in used state
{'status': 'success', 'product': 'nvidia rtx 3060', 'product_status': 'used', 'vendor': 'Parts&Co', 'unit_price': 316.95, 'quantity': 8, 'total_price': 2535.6, 'delivery_available': False, 'delivery_fee': 0, 'expected_delivery_days': None}
🔶 Quick test (non-existing product)--> Summary for: Nintendo Switch 3 in new state
{'status': 'error', 'error_message': 'Product not found or incorrect status, try another product (for example, Samsung Galaxy S22) or status [new, good_state, used, bad_state, for_pieces]'}
🔶 Quick test (existing product) but with a bad state --> Summary for: NVIDIA RTX 3060 in broken state
{'status': 'success', 'product': 'nvidia rtx 3060', 'product_status': 'used', 'vendor': 'Parts&Co', 'unit_price': 321.68, 'quantity': 8, 'total_price': 2573.44, 'delivery_available': False, 'delivery_fee': 0, 'expected_delivery_days': None}
🔶 Quick test (existing product) but with an invalid quantity --> 

## 🧠 Agent re-configuration and re-definition

Having the new tool created, is time to modify the previous agent by adding this function as another tool. For that, inside the instruction I also indicate that the agent needs to use the new tool in order to retrieve all information about the purchase option, as a next step when the agent performs the previous part.

In [24]:
# Market analyst and purchase manager agent creation
product_management = LlmAgent(
    name="product_management",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart product manager, both retrieving information or placing a shopping order.

    1. If user provide product status, use "get_product_summary" tool to get the information related to a product in certain state 
    2. If user don't provide product status or don't mind about it, use "get_product_summary_all_statuses" tool to get infromation related to a product in each state
    3. Check the "status" field in the response of the tool for errors
    4. With these information, use "get_shopping_order_information" tool to get information related to the order of a product given. If no state specified, assume "new".
    5. Return the detailed information obtained explained clearly for the final user, both information of the product and shopping order options.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_product_summary, get_product_summary_all_statuses, get_shopping_order_information],
)

print("✅ Market analyst and purchase manager agent created")
print("🔧 Available tools:")
print("  • get_product_summary - Looks up for a product and given status, retrieving relevant information")
print("  • get_product_summary_all_statuses - Looks up for a product in all statuses, retrieving relevant information")
print("  • get_shopping_order_information - Obtain all details for a purchase options of a product")

✅ Market analyst and purchase manager agent created
🔧 Available tools:
  • get_product_summary - Looks up for a product and given status, retrieving relevant information
  • get_product_summary_all_statuses - Looks up for a product in all statuses, retrieving relevant information
  • get_shopping_order_information - Obtain all details for a purchase options of a product


In [25]:
# Create the new runner in order to run the agent
product_management_runner = InMemoryRunner(agent=product_management)
print("✅ Runner for the market analyst and purchase manager agent created")

✅ Runner for the market analyst and purchase manager agent created


## 👁 Quick model re-testing

In [28]:
print("🔵 Testing with an existing product and state given")
_ = await product_management_runner.run_debug(
    "Can I have some information about the iPhone 14 Pro product in new state?"
)

🔵 Testing with an existing product and state given

 ### Continue session: debug_session_id

User > Can I have some information about the iPhone 14 Pro product in new state?
product_management > I've already provided information about the iPhone 14 Pro in the 'new' state. The actual price is $382.39, and it's currently cheaper than the previous price. The lowest price ever recorded was $342.14, and the highest was $399.84. The vendor is DealHub.

Would you like to know anything else about this product or perhaps place an order?


In [29]:
print("🔵 Testing with an existing product and state given to retrieve shopping order information")
_ = await product_management_runner.run_debug(
    "Give me the shopping order information. I want to buy 4 of these product."
)

🔵 Testing with an existing product and state given to retrieve shopping order information

 ### Continue session: debug_session_id

User > Give me the shopping order information. I want to buy 4 of these product.
product_management > I've already given you the shopping order information for 4 units of the iPhone 14 Pro.

Here's a summary:
- Product: iPhone 14 Pro
- Status: New
- Vendor: DealHub
- Unit Price: $381.39
- Quantity: 4
- Total Price: $1525.56
- Delivery Fee: $4.22
- Expected Delivery: 10 days


In [30]:
print("🔵 Testing with an existing product without state")
_ = await product_management_runner.run_debug(
    "Can I have some information about the google pixel 8 pro product?"
)

🔵 Testing with an existing product without state

 ### Continue session: debug_session_id

User > Can I have some information about the google pixel 8 pro product?


In [31]:
print("🔵 Testing with an existing product and state given")
_ = await product_management_runner.run_debug(
    "Give me the shopping order information"
)

🔵 Testing with an existing product and state given

 ### Continue session: debug_session_id

User > Give me the shopping order information
product_management > I can help you with that! To give you the most accurate shopping order information, please tell me which status you'd like for the Google Pixel 8 Pro (e.g., new, used, good_state, bad_state, or for_pieces). Also, let me know the quantity you wish to purchase.


In [33]:
print("🔵 Testing with an existing product and state given")
_ = await product_management_runner.run_debug(
    "I want to buy 8 of these and in used quality would be fine"
)

🔵 Testing with an existing product and state given

 ### Continue session: debug_session_id

User > I want to buy 8 of these and in used quality would be fine
product_management > I've already provided the shopping order information for 8 units of the Google Pixel 8 Pro in used condition.

Here's a summary:
- Product: Google Pixel 8 Pro
- Status: Used
- Vendor: AppleWorld
- Unit Price: $167.75
- Quantity: 8
- Total Price: $1342.00
- Delivery: Not available


In [35]:
print("🔵 Testing with an existing product and state given")
_ = await product_management_runner.run_debug(
    "And what about Sony WH-1000XM5 new?"
)

🔵 Testing with an existing product and state given

 ### Continue session: debug_session_id

User > And what about Sony WH-1000XM5 new?
product_management > The Sony WH-1000XM5 in new condition is available for $524.17 from DeviceLab. This price is cheaper compared to the previous price, and it's not the lowest or highest it has ever been. The lowest price ever was $498.50, and the highest was $538.63.


In [37]:
print("🔵 Testing with a product that does not exist")
_ = await product_management_runner.run_debug(
    "Give me directly the shop order info for 16 items for the last mentioned item? Use also the state mentioned"
)

🔵 Testing with a product that does not exist

 ### Continue session: debug_session_id

User > Give me directly the shop order info for 16 items for the last mentioned item? Use also the state mentioned
product_management > Here is the shopping order information for 16 new Sony WH-1000XM5 headphones:

*   **Product:** Sony WH-1000XM5
*   **Status:** New
*   **Vendor:** DeviceLab
*   **Unit Price:** $532.90
*   **Quantity:** 16
*   **Total Price:** $8526.40
*   **Delivery Fee:** $2.56
*   **Expected Delivery:** 12 days


# 💡 Improve test using models 

Looking at the previous simple tests, seems obvious that there are not enough in order to see the performance of the model. Because of that, a more complex and suitable way to evaluate the created model is needed.

As we previously learn through the course, real-world conversations are completely dynamic and unpredictable, making the typical fixed test cases used in traditional evaluation useless. 

To handle this problem, I will **create an evaluation test using "User Simulation" tool provided by the ADK** to avoid the static evaluation limitations. By doing this, I'm able to use an agent that dynamically generates user prompts during the evaluation process, simulating a real conversation and handling the unpredictable nature of agent conversations.

## 👉 Create scenario for ADK

The code running inside the notebook is like a "live sandbox", but the ADK CLI doesn't use the interpreter of the notebook. When a new ADK process spawns, ADK requires your code to behave like a real installable Python package (not relying in notebook environment state). In other words, notebook is stateful, dynamic and interactive while ADK is stateless, static and package-based. Because of this, anything developed inside the notebook must eventually be converted into static module files to create a stable, reproducible package hierarchy. 

To achieve that, I need to create the corresponding module files in order to create a full static hierarchy that matches with the current notebook state. With this static scenario created, I'll be able to launch the previously mentioned agent-based tests.

In [38]:
# Create folder hierarchy
os.makedirs("agents", exist_ok=True)
os.makedirs("agents/product_management", exist_ok=True)
os.makedirs("utils", exist_ok=True)
os.makedirs("utils/tools", exist_ok=True)
os.makedirs("product_management_testing", exist_ok=True)

print("✅ Created folder hierarchy for ADK.")

✅ Created folder hierarchy for ADK.


In [39]:
%%writefile utils/tools/tools.py
import sqlite3
import json

DB_NAME = "mock_products.db"

def get_product_summary(product_name: str, status: str):
    """Obtain the current features of a product and status given.

    This tool simulates looking up over internet and find the best option of a product in a certain status and returns the most relevant information about it.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".
        status: The status of the product to find information about it.

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "product": Samsung Galaxy S22,
            "vendor": TechMart,
            "actual_price": 158.25,
            "price_trend_vs_previous": cheaper,
            "is_lowest_price_ever": False,
            "is_highest_price_ever": True,
            "highest_price_ever": 199.99,
            "lowest_price_ever": 155.58
        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """    

    product_name = product_name.lower()
    status = status.lower()
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT product_name, product_vendor, last_checked_prices, status
        FROM products
        WHERE product_name = ? AND status = ?
    """, (product_name, status))

    rows = cursor.fetchall()
    conn.close()

    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found or incorrect status, try another product (for example, Samsung Galaxy S22) or status [new, good_state, used, bad_state, for_pieces]"
        }

    # Convert rows to objects
    entries = []
    for name, vendor, prices_json, st in rows:
        prices = json.loads(prices_json)
        entries.append({
            "product_name": name,
            "product_vendor": vendor,
            "prices": prices,
            "actual_price": prices[-1]
        })

    # Get the cheapest current entry
    cheapest_entry = min(entries, key=lambda x: x["actual_price"])
    prices = cheapest_entry["prices"]

    actual_price = prices[-1]
    previous_price = prices[-2] if len(prices) > 1 else actual_price

    trend = (
        "cheaper" if actual_price < previous_price else
        "expensive" if actual_price > previous_price else
        "same"
    )

    min_price = min(prices)
    max_price = max(prices)

    summary = {
        "status": "success",
        "product": cheapest_entry["product_name"],
        "vendor": cheapest_entry["product_vendor"],
        "actual_price": actual_price,
        "price_trend_vs_previous": trend,
        "is_lowest_price_ever": actual_price == min_price,
        "is_highest_price_ever": actual_price == max_price,
        "highest_price_ever": max_price,
        "lowest_price_ever": min_price
    }

    return summary

def get_product_summary_all_statuses(product_name: str):
    """Obtain the current features of a product and status given.

    This tool simulates looking up over internet and find the best option of a product in all status options and returns the most relevant information about it.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "new":{
                "product": Samsung Galaxy S22,
                "vendor": TechMart,
                "actual_price": 158.25,
                "price_trend_vs_previous": cheaper,
                "is_lowest_price_ever": False,
                "is_highest_price_ever": True,
                "highest_price_ever": 199.99,
                "lowest_price_ever": 155.58
            }
            "used": {
            ...
            }

        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """    

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    product_name = product_name.lower()
    
    cursor.execute("""
        SELECT product_name, product_vendor, last_checked_prices, status
        FROM products
        WHERE product_name = ?
    """, (product_name,))

    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found, try another one (for example, Samsung Galaxy S22)"
        }

    # Organize rows by status
    status_groups = {}
    for name, vendor, prices_json, status in rows:
        prices = json.loads(prices_json)

        entry = {
            "product_name": name,
            "product_vendor": vendor,
            "prices": prices,
            "actual_price": prices[-1]
        }

        status_groups.setdefault(status, []).append(entry)

    summaries = {"status": "success"}

    # Build summary per status
    for status, entries in status_groups.items():
        # pick entry with cheapest actual price
        cheapest_entry = min(entries, key=lambda x: x["actual_price"])
        prices = cheapest_entry["prices"]

        actual_price = prices[-1]
        previous_price = prices[-2] if len(prices) > 1 else actual_price

        trend = (
            "cheaper" if actual_price < previous_price else
            "expensive" if actual_price > previous_price else
            "same"
        )

        min_price = min(prices)
        max_price = max(prices)

        summaries[status] = {
            "product": cheapest_entry["product_name"],
            "vendor": cheapest_entry["product_vendor"],
            "actual_price": actual_price,
            "price_trend_vs_previous": trend,
            "is_lowest_price_ever": actual_price == min_price,
            "is_highest_price_ever": actual_price == max_price,
            "lowest_price_ever": min_price,
            "highest_price_ever": max_price
        }

    return summaries

def get_shopping_order_information(product_name: str, status: str, quantity: int):

    """Place a shopping order of a product and swith a number of items.

    This tool simulates a product order, calculating the price, the delivery fee and the delivery expected time.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".
        status: The status of the product to place the order.
        quantity: The quantity of items to purchase.

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "item_status": new
            "product": Samsung Galaxy S22,
            "vendor": TechMart,
            "unit_price": 158.25,
            "quantity": quantity,
            "total_price": total_price,
            "delivery_available": "True",
            "delivery_fee": 0,
            "expected_delivery_days": 3
        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """  

    product_name = product_name.lower()
    status = status.lower()        

    if not isinstance(quantity, int) or quantity < 1:
        return {
            "status": "error", 
            "error_message": "Quantity needs to be a valid number and at greather than 0"
        }

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT last_checked_prices, delivery_available, delivery_days,
               delivery_fee, product_vendor
        FROM products
        WHERE product_name = ? AND status = ?
    """, (product_name, status))

    rows = cursor.fetchall()
    conn.close()

    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found, try another one (for example, Samsung Galaxy S22)"
        }

    # Convert DB rows into structured objects
    entries = []
    for prices_json, delivery_av, delivery_days, delivery_fee, vendor in rows:
        prices = json.loads(prices_json)

        entries.append({
            "actual_price": prices[-1],
            "delivery_available": delivery_av == 1,
            "delivery_days": delivery_days,
            "delivery_fee": delivery_fee,
            "vendor": vendor,
        })

    # Select the cheapest entry by current price
    best = min(entries, key=lambda x: x["actual_price"])

    # Compute totals
    unit_price = best["actual_price"]
    total_price = round(unit_price * quantity, 2)

    return {
        "status": "success",
        "product": product_name,
        "product_status": status,
        "vendor": best["vendor"],
        "unit_price": unit_price,
        "quantity": quantity,
        "total_price": total_price,
        "delivery_available": best["delivery_available"],
        "delivery_fee": best["delivery_fee"] if best["delivery_available"] else 0,
        "expected_delivery_days": best["delivery_days"] if best["delivery_available"] else None
    }

Writing utils/tools/tools.py


In [40]:
%%writefile utils/database.py
import sqlite3
import os

DB_NAME = "mock_products.db"
DB_PATH = f"/kaggle/working/{DB_NAME}"

def get_connection():
    return sqlite3.connect(DB_PATH)

# Database creation and definition
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_name TEXT,
            product_vendor TEXT,
            product_specs TEXT,
            last_checked_prices TEXT,
            delivery_available INTEGER,
            delivery_days INTEGER,
            delivery_fee REAL,
            status TEXT
        );
    """)

    conn.commit()
    conn.close()


# Function to insert a single product into database
def insert_product(entry: str):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO products 
        (product_name, product_vendor, product_specs, last_checked_prices,
         delivery_available, delivery_days, delivery_fee, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        entry["product_name"],
        entry["product_vendor"],
        entry["product_specs"],
        json.dumps(entry["last_checked_prices"]),
        1 if entry["delivery_available"] else 0,
        entry["delivery_days"],
        entry["delivery_fee"],
        entry["status"]
    ))

    conn.commit()
    conn.close()



# Function to fill the database with invented data
def seed_database():
    devices = [
        ("iPhone 13", "128GB, OLED display, A15 Bionic"),
        ("iPhone 14 Pro", "256GB, ProMotion 120Hz"),
        ("Samsung Galaxy S22", "128GB, Snapdragon 8 Gen 1"),
        ("Samsung Galaxy S23 Ultra", "512GB, S-Pen"),
        ("Google Pixel 7", "128GB, Tensor G2"),
        ("Google Pixel 8 Pro", "256GB, Tensor G3"),
        ("MacBook Air M1", "8GB RAM, 256GB SSD"),
        ("MacBook Air M2", "16GB RAM, 512GB SSD"),
        ("MacBook Pro M3", "18GB RAM, 1TB SSD"),
        ("iPad Pro M2", "11-inch, 128GB"),
        ("Dell XPS 13", "16GB RAM, 512GB SSD"),
        ("Lenovo ThinkPad X1 Carbon", "16GB RAM, 1TB SSD"),
        ("Asus ROG Zephyrus G14", "RTX 4060, 16GB RAM"),
        ("Sony WH-1000XM4", "ANC headphones"),
        ("Sony WH-1000XM5", "ANC headphones"),
        ("Bose QC45", "Noise cancelling headphones"),
        ("Logitech MX Master 3", "Ergonomic mouse, USB-C"),
        ("Logitech G Pro X", "Gaming headset"),
        ("Razer Viper Ultimate", "Wireless gaming mouse"),
        ("SteelSeries Arctis 7", "Wireless gaming headset"),
        ("Samsung 970 EVO Plus 1TB", "NVMe SSD"),
        ("Samsung 990 Pro 2TB", "NVMe SSD"),
        ("Crucial MX500 1TB", "SATA SSD"),
        ("WD Black SN850X", "NVMe SSD"),
        ("NVIDIA RTX 3060", "12GB GDDR6"),
        ("NVIDIA RTX 4070", "12GB GDDR6X"),
        ("NVIDIA RTX 4090", "24GB GDDR6X"),
        ("AMD Radeon 6800 XT", "16GB GDDR6"),
        ("AMD Ryzen 7 5800X3D", "8-core CPU"),
        ("Intel Core i9 13900K", "24-core CPU"),
        ("Oculus Quest 2", "VR headset"),
        ("Meta Quest 3", "Next-gen VR headset"),
        ("Amazon Kindle Paperwhite", "E-ink display"),
        ("Kindle Oasis", "Premium e-reader"),
        ("LG C2 OLED 55", "4K OLED TV"),
        ("Samsung Odyssey G7", "27'' QHD 240Hz monitor"),
        ("LG 27GP850", "27'' 165Hz gaming monitor"),
        ("Dell UltraSharp U2723QE", "27'' 4K IPS display"),
    ]

    statuses = {
        "new": (200, 2200),
        "good_state": (120, 1500),
        "used": (80, 1000),
        "bad_state": (20, 400),
        "for_pieces": (5, 100),
    }

    vendors = [
        "TechStore", "GigaBuy", "SecondLifeElectronics", "SoundHub", "AudioResale",
        "ScrapTech", "ReTech", "Parts&Co", "GPUPlanet", "AppleWorld",
        "MobileHub", "MegaHardware", "CompuTrade", "ElectroZone", "HyperTech",
        "SmartBuy", "DealHub", "DiscountTech", "FutureElectronics", "NeoTech",
        "TechMart", "ScreenShop", "VRPlanet", "DigitalStation", "ElectronixPro",
        "ComputerPlaza", "FireTech", "SmartGizmos", "DeviceLab", "TechTrade",
        "EuTech", "DigitalDen", "SiliconWorld", "GizmoDepot", "TechieStore",
        "GadgetHouse", "ElectronHub", "ProTechDeals", "ChipMasters"
    ]

    all_entries = []

    for name, specs in devices:
        
        name = name.lower()
        
        for status, price_range in statuses.items():
            for _ in range(3):  
                base = random.uniform(*price_range)

                entry = {
                    "product_name": name,
                    "product_vendor": random.choice(vendors),
                    "product_specs": specs,
                    "last_checked_prices": [
                        round(base + random.uniform(-15, 15), 2),
                        round(base + random.uniform(-15, 15), 2)
                    ],
                    "delivery_available": random.choice([True, False]),
                }

                if entry["delivery_available"]:
                    entry["delivery_days"] = random.randint(1, 14)
                    entry["delivery_fee"] = round(random.uniform(0, 40), 2)
                else:
                    entry["delivery_days"] = -1
                    entry["delivery_fee"] = -1

                entry["status"] = status
                all_entries.append(entry)

    for e in all_entries:
        insert_product(e)

    print(f"Inserted {len(all_entries)} entries into database.")

Writing utils/database.py


In [41]:
%%writefile utils/tools/__init__.py
from .tools import (
    get_product_summary,
    get_product_summary_all_statuses,
    get_shopping_order_information,
)

Writing utils/tools/__init__.py


In [42]:
%%writefile agents/product_management/agent.py
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.models.google_llm import Gemini
from google.genai import types

# Import your tool functions (they will exist in utils/tools.py)
from utils.tools import (
    get_product_summary,
    get_product_summary_all_statuses,
    get_shopping_order_information,
)

# Wrap functions as ADK tools
get_product_summary_tool = FunctionTool(get_product_summary)
get_product_summary_all_statuses_tool = FunctionTool(get_product_summary_all_statuses)
get_shopping_order_information_tool = FunctionTool(get_shopping_order_information)

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

agent = LlmAgent(
    name="product_management",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart product manager, both retrieving information or placing a shopping order.

    1. If user provide product status, use "get_product_summary" tool to get the information related to a product in certain state 
    2. If user don't provide product status or don't mind about it, use "get_product_summary_all_statuses" tool to get infromation related to a product in each state
    3. Check the "status" field in the response of the tool for errors
    4. With these information, use "get_shopping_order_information" tool to get information related to the order of a product given. If no state specified, assume "new".
    5. Return the detailed information obtained explained clearly for the final user, both information of the product and shopping order options.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_product_summary, get_product_summary_all_statuses, get_shopping_order_information],
)

#__all__ = ["agent"]


Writing agents/product_management/agent.py


In [43]:
%%writefile agents/product_management/__init__.py
from .agent import agent

Writing agents/product_management/__init__.py


In [44]:
%%writefile utils/__init__.py
#from .utils import utils

Writing utils/__init__.py


## ⚙️ Evaluation tests setup

Now, is time to create some evaluation cases that can check how our agent works in different scenarios. Let's define different cases to test basic functions.

Once created, I also configure the evaluation parameters in order to take control of the way of applying these evaluation exams. For example, I use the temperature, top_k and top_p parameters to get randomness, predictable and consistent responses; or the parameter max_response_length to avoid receiving too long responses.

In [45]:
# Create test scenarios for a better understanding of the performance of the model
scenarios_test_cases = {
  "scenarios": [
    # 1 - Normal flow - User asks for a information with an existing product but without state
    {
      "starting_prompt": "Can you retrieve me info about the Lenovo ThinkPad X1 Carbon?",
      "conversation_plan": "Ask again but for these product in a good_state. \
                            After you get the result, tell the agent to receive the shopping information for the same state and 19 units."
    },

    # 2 - Normal flow - User asks for a information with an existing product and state
    {
      "starting_prompt": "Can you retrieve me info about the Meta Quest 3 in new state?",
      "conversation_plan": "Ask again but for these product in a for_pieces state. \
                            After you get the result, tell the agent to obtain the shopping information without more information. \
                            When agent asked for units and state, say 58 units and same state as you asked before for information"
    },

    # 3 - Asking for a non-existing product
    {
      "starting_prompt": "What can you tell me about the Nintendo Switch 3?",
      "conversation_plan": "Ask again indicating that you want it in a bad_state state. \
                            Regardless the response, try to ask about the shopping information about it."
    },

    # 4 — Request shopping order before any product info (should trigger tool order logic)
    {
      "starting_prompt": "Can you tell me if I can order 12 units of the Logitech G Pro X?",
      "conversation_plan": "If agent asks for product status, say 'new'. \
                            After product info and order info are returned, ask again but for 'for_pieces' state."
    },

    # 5 — User provides a nonexistent state (should cause error-handling path)
    {
      "starting_prompt": "Can I get product info about the Samsung Galaxy S23 Ultra in ultra_used state?",
      "conversation_plan": "If the agent returns an error, ask what valid states exist. \
                            Then ask for shopping info assuming the 'new' state and 2 units."
    },

    # 6 — Multi-product cross-check (with only one existing)
    {
      "starting_prompt": "Tell me the info for both the Lenovo ThinkPad X1 Carbon and the HP Spectre x360.",
      "conversation_plan": "When the agent chooses one, ask for the other in good state. \
                            Then request shopping information for both, 2 units each."
    },

    # 7 — User provides no product name at first (agent should ask)
    {
      "starting_prompt": "Can you check the product information for me?",
      "conversation_plan": "When the agent asks, specify 'Sony WH-1000XM5' in new state. \
                            Then ask for shopping information without specifying units. \
                            When asked, answer that you want 10 units."
    },

    # 8 — Asking only for shopping info (forces agent to perform info retrieval first)
    {
      "starting_prompt": "I want to buy 25 units of the Logitech MX Master 3 mouse.",
      "conversation_plan": "If the agent asks for the state, respond that you want information for all states first. \
                            After agent gives product info, request the shopping info for the 'good' state."
    },

    # 9 — Product exists but user asks for unsupported state first
    {
      "starting_prompt": "Give me info about the AMD Radeon 6800 XT in broken status.",
      "conversation_plan": "After the error, say you want 'for_pieces' instead. \
                            Then ask for shopping information for the same state and 14 units."
    },

    # 10 — Stress test: user changes mind mid-conversation
    {
      "starting_prompt": "Can you get information about the Oculus Quest 2?",
      "conversation_plan": "After answer, say you actually want info in all states. \
                            Then request shopping info but change the units mid-process: first say 100 units, then say actually 35 units."
    },
  ]
}

with open("product_management_testing/product_management_test_cases.json", "w") as f:
    json.dump(scenarios_test_cases, f, indent=2)

print("✅ Created and stored some test cases")

✅ Created and stored some test cases


In [46]:
# Create session input
session_input = {
  "app_name": APP_NAME,
  "user_id": USER_ID
}
with open("product_management_testing/product_management_session_input.json", "w") as f:
    json.dump(session_input, f, indent=2)
    
print("✅ Created and stored the session parameters")


# Create a configuration dict with all criteria and agent simulation for testing purpose
eval_configuration = {
  "criteria": {
    "hallucinations_v1": {
      "threshold": 0.5,
      "evaluate_intermediate_nl_responses": True
    }
  },
  "user_simulator_config": {
    "model": "gemini-2.5-flash",
    "persona": "A concise and pragmatic customer asking for product information \
                 and shopping availability. Responds directly, never behaves \
                 like an assistant, and follows the conversation plan strictly.",
    "response_constraints": {
      "max_response_length": 150,
      "avoid_repeating_information": True,
      "stay_within_conversation_plan": True
    },
    "tool_response_behavior": {
      "handle_tool_errors": "acknowledge_and_adjust_request",
      "respond_to_tool_prompts": True
    },
    "model_configuration": {
      "temperature": 0.2, 
      "top_p": 1.0, 
      "top_k": 40,
      "thinking_config": {
        "include_thoughts": True,
        "thinking_budget": 10240
      }
    },
    "max_allowed_invocations": 20
  }
}
with open("product_management_testing/product_management_configuration.json", "w") as f:
    json.dump(eval_configuration, f, indent=2)

print("✅ Created and stored the configuration parameters for the evaluation")

✅ Created and stored the session parameters
✅ Created and stored the configuration parameters for the evaluation


## 🔥 Launch improved tests

In [47]:
!adk eval_set create agents/product_management eval_set_with_scenarios
print("✅ Already evaluation set created for the Product Management agent")

!adk eval_set add_eval_case agents/product_management eval_set_with_scenarios \
    --scenarios_file product_management_testing/product_management_test_cases.json \
    --session_input_file product_management_testing/product_management_session_input.json
print("✅ Already added the custom test to the evaluation set previously created for the Product Management agent")

INFO:google_adk.google.adk.evaluation.local_eval_sets_manager:Creating eval set file `/kaggle/working/agents/product_management/eval_set_with_scenarios.evalset.json`
INFO:google_adk.google.adk.evaluation.local_eval_sets_manager:Eval set file doesn't exist, we will create a new one.
Eval set 'eval_set_with_scenarios' created for app 'product_management'.
✅ Already evaluation set created for the Product Management agent
Eval case 'ad28c06f' added to eval set 'eval_set_with_scenarios'.
Eval case '36a19961' added to eval set 'eval_set_with_scenarios'.
Eval case '96714fce' added to eval set 'eval_set_with_scenarios'.
Eval case '70464853' added to eval set 'eval_set_with_scenarios'.
Eval case '7b7fecd6' added to eval set 'eval_set_with_scenarios'.
Eval case 'e2e1dcfb' added to eval set 'eval_set_with_scenarios'.
Eval case '991676d7' added to eval set 'eval_set_with_scenarios'.
Eval case 'd30f3014' added to eval set 'eval_set_with_scenarios'.
Eval case 'e1c6cd0d' added to eval set 'eval_set_w

In [48]:
!PYTHONPATH=/kaggle/working adk eval agents/product_management \
    --config_file_path product_management_testing/product_management_configuration.json \
    eval_set_with_scenarios \
    --print_detailed_results
print("✅ Already performed custom test with the Product Management agent")

/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/local_eval_service.py:79: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
Using evaluation criteria: criteria={'hallucinations_v1': BaseCriterion(threshold=0.5, evaluate_intermediate_nl_responses=True)} user_simulator_config=BaseUserSimulatorConfig(model='gemini-2.5-flash', persona='A concise and pragmatic customer asking for product information                  and shoppi

## ❇️ Main test insights 

For each test the main insights are:

- 1 (🟢) Normal flow - User asks for a information with an existing product but without state

This first case works perfectly, returning the information for all product states and then generating the purchase order based on the state and quantity specified.

- 2 (🟡) Normal flow - User asks for a information with an existing product and state

The second case follows the expected flow, returning the information correctly, but there is a failure in the product state check. The specified state is valid, but the checks are incorrect. There is a discrepancy in the check for this case.

- 3 (🟢) Asking for a non-existing product

When attempting to use a product that does not exist in the database, the system correctly indicates that the product does not exist in both attempts to request it.

- 4 (🟡) Request shopping order before any product info (should trigger tool order logic)

The agent understands the conversation, realizes it needs the product status, and asks correctly. It then makes the call correctly and returns the information, but with a delay. It seems as if the two agents were out of sync in this test. Both appear to be working correctly, but with a delay.

- 5 (🟢) User provides a nonexistent state (should cause error-handling path)

This conversation works perfectly. Upon realizing that the status is invalid, it asks for a valid one, and upon receiving it, it makes the call to the tool and returns the information exactly as requested.

- 6 (🟢) Multi-product cross-check (with only one existing)

When asking for two products (one of which does not exist), it correctly indicates that one of the products does not exist and provides information for the other. When asked again, it correctly responds that the other product does not exist, maintaining consistency.

- 7 (🟢) User provides no product name at first (agent should ask)

This conversation also follows the expected structure. It asks for the product name upon realizing that it was not provided and follows the correct flow by asking for the status and then the quantity for the purchase order, completing the expected flow.

- 8 (🟡) Asking only for shopping info (forces agent to perform info retrieval first)

This fails simply because the call also includes the word "mouse," which doesn't match the information in the database.

- 9 (🟢) Product exists but user asks for unsupported state first

The conversation follows the normal flow, indicating that the product doesn't exist and the available states. Upon receiving a valid state, it correctly returns the purchase information.

- 10 (🟡) Stress test: user changes mind mid-conversation

In this conversation, the response seems to freeze. The call to `get_product_summary_all_statuses` is made correctly, indicating that the user is aware of the request and wants to obtain information for all states, but doesn't provide the response to the "user agent." Then, the user confuses wanting all information states with purchase states, which the application cannot do by design, and responds by stating that it cannot do so due to its own limitations.

**By conducting tests in this way, it is much easier to observe the weaknesses of our model**. For example, we have seen that better validation of the function input with the product name is needed (specifying the product name and type, for example, Logitech... mouse, can cause a problem since the database entry does not include the word "mouse"), and the product status validation also needs improvement and consistency.

# 🤖 Improve agent: Long-Running Operations

The use of Long-Running Operations (Human-in-the-Loop) is a crucial feature in a huge variety of scenarios. The possibility to pause the agent when some complex operations are running, making humans able to approve or deny this operation, could save the agent (or any user that uses the agent) from performing a non-desired action. Examples could be asking before deleting a huge amount of rows in a database or before making too expensive transactions.

In this specific scenario, a very clear use case appears where this feature can be used. Currently, the proposed agent places a purchase order regardless the whole amount of the purchase, because of its own nature. 

Think in this case. The user sends this message "Give me the purchase order to buy Nvidia RTX 3060. I want 100", where the user wanted to order 10 but wrongly pressed 0 one more time, sending it quickly. At this moment, the user does not have any way to stop or cancel the order, and the purchase order will be placed.

To solve this problem, it's time to add this feature to the current agent. When the shopping order price is more than 1000$, the agent will be paused and waiting for user approval or denied, and only placed if the user explicitly approves it.

By doing this, the purchase orders that are relatively small will be placed at the moment, where only the sensible decisions will require human approval.

## 🛠 Tool improvement

The tool improved:

- get_shopping_order_information_with_hitl(product_name: str, status: str, quantity: int, tool_context: ToolContext) -> The tool's core functionality remains the same (it receives the product name, status, and quantity to be purchased, and places a purchase order). However, now that the tool has all the relevant information, it only places purchase orders if the price is less than $1,000. If the price is higher, the tool requests human approval and pauses until it receives a response. The scenarios handled are:

**Scenario 1: Cheap order (<= 1000$)**: Returns immediately with auto-approved status. 

**Scenario 2: Large order - Initial step (> 1000$)**: The agent execution pauses after request human approval, retrieving pending status.

**Scenario 3: Large order - Final step (> 1000$)**: The agent checks human decision and acts accordingly,retrieving success or error status.

In [49]:
# Method to return all available information of purchase a product (simulates placing a purchase order) adding the human approval option
def get_shopping_order_information_with_hitl(product_name: str, status: str, quantity: int, tool_context: ToolContext):

    """Place a shopping order of a product and swith a number of items. Requires approval if the entire order is more expensive than $1000.

    This tool simulates a product order, calculating the price, the delivery fee and the delivery expected time.

    Args:
        product_name: The name of the product that you are looking for. Examples: "Samsung Galaxy S22" or "Intel Core i9 13900K".
        status: The status of the product to place the order.
        quantity: The quantity of items to purchase.

    Returns:
        Dictionary with status and product information.
        Success: { 
            "status": "success"
            "item_status": new
            "product": Samsung Galaxy S22,
            "vendor": TechMart,
            "unit_price": 158.25,
            "quantity": quantity,
            "total_price": total_price,
            "delivery_available": "True",
            "delivery_fee": 0,
            "expected_delivery_days": 3
        }
        Error: {"status": "error", "error_message": "Product not found, try another one"}
    """  

    product_name = product_name.lower()
    status = status.lower()        

    if not isinstance(quantity, int) or quantity < 1:
        return {
            "status": "error", 
            "error_message": "Quantity needs to be a valid number and at greather than 0"
        }
        

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT last_checked_prices, delivery_available, delivery_days,
               delivery_fee, product_vendor
        FROM products
        WHERE product_name = ? AND status = ?
    """, (product_name, status))

    rows = cursor.fetchall()
    conn.close()

    apply_price_fluctuation() # To simulate market drift each time this function/tool is invoked

    if not rows:
        return {
            "status": "error", 
            "error_message": "Product not found or incorrect status, try another product (for example, Samsung Galaxy S22) or status [new, good_state, used, bad_state, for_pieces]"
        }

    # Convert DB rows into structured objects
    entries = []
    for prices_json, delivery_av, delivery_days, delivery_fee, vendor in rows:
        prices = json.loads(prices_json)

        entries.append({
            "actual_price": prices[-1],
            "delivery_available": delivery_av == 1,
            "delivery_days": delivery_days,
            "delivery_fee": delivery_fee,
            "vendor": vendor,
        })

    # Select the cheapest entry by current price
    best = min(entries, key=lambda x: x["actual_price"])

    # Compute totals
    unit_price = best["actual_price"]
    total_price = round(unit_price * quantity, 2)

    # Scenario 1: Cheap order (<= 1000$) // Human interaction for approval not required
    if total_price < 1000: 
        return {
            "status": "approved",
            "product": product_name,
            "product_status": status,
            "vendor": best["vendor"],
            "unit_price": unit_price,
            "quantity": quantity,
            "total_price": total_price,
            "delivery_available": best["delivery_available"],
            "delivery_fee": best["delivery_fee"] if best["delivery_available"] else 0,
            "expected_delivery_days": best["delivery_days"] if best["delivery_available"] else None
        }

    # Scenario 2: Large order - Initial step (> 1000$) // Human interaction for approval required (pause here the execution until the tool is called again)
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Expensive order: Purchase order of {quantity} {product_name} costs {total_price} (which exceeds the $1000 limit). Do you want to approve?",
            payload={"quantity": quantity, "product_name": product_name, "total_price": total_price},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Purchase order of {quantity} {product_name} requires approval. The cost of {total_price} exceed the 1000$ security limit.",
        }

    # Scenario 3: Large order - Final step (> 1000$) // The tool is called after human interaction
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "product": product_name,
            "product_status": status,
            "vendor": best["vendor"],
            "unit_price": unit_price,
            "quantity": quantity,
            "total_price": total_price,
            "delivery_available": best["delivery_available"],
            "delivery_fee": best["delivery_fee"] if best["delivery_available"] else 0,
            "expected_delivery_days": best["delivery_days"] if best["delivery_available"] else None
        }
    else:
        return {
            "status": "rejected"
        }   

print("✅ Tool function to place purchase orders created with human approval (required if the order is too expensive).")

✅ Tool function to place purchase orders created with human approval (required if the order is too expensive).


## 🧠 Agent final definition

After updating the new tool, it's time to improve the previous agent by adding this function as an additional tool, and the previous agent as a tool as well. The instructions have also been improved to ensure the tool (the agent as a tool) is used correctly, requiring human approval if necessary.

In [50]:
# Market analyst and purchase manager agent creation
product_management_complex = LlmAgent(
    name="product_management",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart product manager, both retrieving information or placing a shopping order.

    1. If user provide product status, use "market_analyst_agent" tool to get the information related to a product in any state or a given state.
    2. Check the "status" field in the response of the tool for errors.
    3. With these information, use "get_shopping_order_information" tool to get information related to the order of a product given. If no state specified, assume "new".
    4. If the order status is 'pending', inform the user that approval is required with the message obtained.
    5. Return the detailed information clearly for the final user, both information of the product and shopping placement.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[AgentTool(market_analyst_agent), FunctionTool(func=get_shopping_order_information_with_hitl)],
)

print("✅ Complex product and purchase management agent created.")
print("🔧 Available tools:")
print("  • market_analyst_agent - Looks up for a product and optional status, retrieving relevant information.")
print("  • get_shopping_order_information - Manage a purchase placement with the user with optional approval requirement.")

✅ Complex product and purchase management agent created.
🔧 Available tools:
  • market_analyst_agent - Looks up for a product and optional status, retrieving relevant information.
  • get_shopping_order_information - Manage a purchase placement with the user with optional approval requirement.


At this point, the created agent still stateless and the agent don't have an way to remember what it was doing. To avoid this problem:

- I need to create a new app with the resumability configuration, wrapping the agent inside in order to add a persistence layer (handle states)
- I need to create one runner that runs properly the resumable app.

In [51]:
product_management_complex_app = App(
    name="product_management_complex_app",
    root_agent=product_management_complex,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app for human-in-the-loop functionality created.")

✅ Resumable app for human-in-the-loop functionality created.


In [52]:
session_service = InMemorySessionService()

# Create runner with the resumable app
product_management_complex_runner = Runner(
    app=product_management_complex_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner with the resumable app and session created.")

✅ Runner with the resumable app and session created.


## ⚙️ Setup functions and workflow

Having the runner ready, is time to test the model to know if the human interaction works correctly (and only when needed).

For that, and following the example of the course, let's create a function that simulates the whole workflow of a real conversation between a user and the agent, adapting the functions to this specific scenario.

In [53]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None


def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")


def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


async def run_purchase_workflow(query: str, auto_approve: bool = True):
    """Runs a purchase workflow with approval handling.

    Args:
        query: User's purchase request
        auto_approve: Whether to auto-approve expensive orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="product_management_complex_app", user_id=USER_ID, session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in product_management_complex_runner.run_async(
        user_id=USER_ID, session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in product_management_complex_runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Functions to simulate a complete workflow with human interaction ready.")

✅ Functions to simulate a complete workflow with human interaction ready.


## 👁 Model human-interaction testing

In [55]:
print("🔵 Testing query with cheap shopping order (less than 1000$, human approved not required).")
await run_purchase_workflow("Place a shopping order of 1 NVIDIA RTX 3060")

🔵 Testing query with cheap shopping order (less than 1000$, human approved not required).

User > Place a shopping order of 1 NVIDIA RTX 3060

Agent > Here is the information for your order:

**Product:** NVIDIA RTX 3060
**Vendor:** GadgetHouse
**Quantity:** 1
**Unit Price:** $540.66
**Total Price:** $540.66
**Delivery Available:** False
**Delivery Fee:** $0
**Expected Delivery Days:** None

The order has been approved.



In [56]:
print("🔵 Testing query with expensive shopping order (more than 1000$, human approved required). Simulate user approves the order placement.")
await run_purchase_workflow("Place a shopping order of 27 NVIDIA RTX 3060")

🔵 Testing query with expensive shopping order (more than 1000$, human approved required). Simulate user approves the order placement.

User > Place a shopping order of 27 NVIDIA RTX 3060

⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > The order for 27 NVIDIA RTX 3060 has been placed. The total price is $14259.24. The vendor is GadgetHouse. Delivery is not available for this order.



In [59]:
print("🔵 Testing query with expensive shopping order (more than 1000$, human approved required). Simulate user not approves the order placement.")
await run_purchase_workflow("Place a shopping order of 27 NVIDIA RTX 3060", auto_approve=False)

🔵 Testing query with expensive shopping order (more than 1000$, human approved required). Simulate user not approves the order placement.

User > Place a shopping order of 27 NVIDIA RTX 3060

⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > I'm sorry, but I cannot fulfill this order. The order has been rejected.



In [60]:
print("🔵 Testing query with a non existing product.")
await run_purchase_workflow("Place a shopping order of 27 Nintendo Switch 3")

🔵 Testing query with a non existing product.

User > Place a shopping order of 27 Nintendo Switch 3

Agent > I am sorry, I cannot fulfill this request. The product "Nintendo Switch" was not found or the status is incorrect. Please try another product, for example, "Samsung Galaxy S22", or one of the following statuses: "new", "good_state", "used", "bad_state", "for_pieces".



# 🎯 Conclusion

Throughout this notebook, we have presented an AI agent-based solution that addresses a real need for many companies, and is also applicable to end users.

We have simulated a real-world environment by creating databases and methods that simulate the use of an API or similar. By using real APIs and search engines, this solution could be integrated into real-world environments.

After developing the final agent, a solution focused on advanced model evaluation was presented, compared to the one shown in the course. This solution used an agent to simulate conversations with the created agent, allowing for a more effective evaluation.

Related to next steps, ideas like looking for a type of product (instead of product itself), looking for the best products of a specific vendor or filtering directly the options that don't have a delivery fee could be well-fitted improvements.

As a summary, I applied some of the knowledge acquired during this short and intense course to a "simulated" real environment, having an agent that helps to avoid (or at least reduce) the initial commented problem.